In [26]:
import os
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import re
import time

In [27]:
def extract_data(url, target_data_count=20, output_csv='extracted_data.csv'):
    print(f"Opening page: {url}")  # Print progress as soon as the page starts loading
    
    chrome_options = Options()
    chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration (useful for some environments)
    webdriver_path = r"C:\users\asus\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"

    # Create a Service object
    service = Service(webdriver_path)
    
    # Initialize the WebDriver using the Service object
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, "lxml")
        data = []

        # Scroll down to load more content
        last_height = driver.execute_script("return document.body.scrollHeight")
        collected_count = 0  # Counter for the number of collected items

        while collected_count < target_data_count:
            # Find the articles on the page
            articles = soup.find_all('article')
            for article in articles:
                title_author = article.find('div', {'class': 'fusion-post-content post-content'})
                a_title = title_author.find('a')
                author_name = title_author.find('p')
                title_link = a_title['href'] if a_title else None 


                year_prize_a = article.find('div', {'class' : "fusion-meta-info"})
                year_prize = year_prize_a.find_all('a')
                year = year_prize[1]
                prize = year_prize[2]



                if a_title and author_name:
                    data.append([a_title.text.strip(), author_name.text.strip(), year.text.strip(), prize.text.strip(), title_link])  # Append both title and author to the list
                    collected_count += 1

                # Stop if we have collected the target amount
                if collected_count >= target_data_count:
                    break

            # If we've collected enough data, exit the loop
            if collected_count >= target_data_count:
                break

            # Scroll down to the bottom of the page to load more content
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait for the new content to load (adjust timeout as needed)
            time.sleep(3)  # Wait for 3 seconds for the page to load more data

            # Check if the page height is increased (indicating more content was loaded)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  # No more new content, exit the loop
            last_height = new_height  # Update the last height for the next iteration

    except Exception as e:
        print(f"Error scraping {url}: {e}")  # Log errors during scraping
        data = []
    finally:
        driver.quit()
    
    print(f"Finished scraping: {url}")  # Notify when scraping for this URL is complete
    print(f"Collected {collected_count} items.")
    
    # Save data to CSV
    save_to_csv(data, output_csv)
    
    return data

def save_to_csv(data, filename):
    """Saves the collected data to a CSV file."""
    try:
        with open(filename, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Title', 'Author', 'Year', 'Type of Prize', 'url'])  # Write the header row with both Title and Author
            writer.writerows(data)  # Write the collected data
        print(f"Data saved to {filename}")
    except Exception as e:
        print(f"Error saving data to CSV: {e}")

# Example usage
extracted_data = extract_data("https://laskerfoundation.org/all-awards-winners/")
print(f"Total data collected: {len(extracted_data)}")


Opening page: https://laskerfoundation.org/all-awards-winners/
Finished scraping: https://laskerfoundation.org/all-awards-winners/
Collected 20 items.
Data saved to extracted_data.csv
Total data collected: 20


In [28]:
# # import required libraries
# from kafka import KafkaProducer
# import time
# from datetime import datetime

In [29]:
# # Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
# kafka_broker = 'localhost:29092'
# producer = KafkaProducer(
#     bootstrap_servers=[kafka_broker],
#     linger_ms=5000,  # Increased linger time
#     max_block_ms=60000,  # Increase the max block time (default: 60000 ms = 1 minute)
# )